In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
import string
import warnings
warnings.filterwarnings('ignore')
from nltk.stem import PorterStemmer
stemming = PorterStemmer()
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from wordcloud import WordCloud
from matplotlib.colors import ListedColormap

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

ModuleNotFoundError: No module named 'wordcloud'

## Loading the data

In [ ]:
df = pd.read_csv('train_set.csv')

In [ ]:
df.head()

## Exploratory Data Analysis

In [ ]:
df.shape

In [ ]:
df.info()

#### Missing Value

In [ ]:
df.isnull().sum()

## Lang_id Features

In [ ]:
df.lang_id.describe()

In [ ]:
#plot a bar graph 
plt.figure(figsize=(12,8)) #set fig size
base_color = sns.color_palette()[0] #set base color of the plot
type_order = df['lang_id'].value_counts().index # set the order of the sentiment: from highest to lowest
ax=sns.countplot(data=df, x='lang_id', color=base_color, order=type_order);
ax.bar_label(ax.containers[0]);# set each bar label
plt.title('Counts of each lang_id');

## Data Engineering

In [ ]:
#Function to remove punctuation
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [ ]:
df['clean_text'] = df['text'].apply(remove_punctuation)

In [ ]:
df.head(10)

#### Remove special punctuation

In [ ]:
# remove special characters, numbers and punctuations
df['clean_text'] = df['clean_text'].str.replace("[^a-zA-Z#]", " ")
df.head(10)

## Model Building

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# Fit label encoder and return encoded labels
y = le.fit_transform(df['lang_id'])

In [ ]:
y

In [ ]:
lang_labels = list(le.classes_)

In [ ]:
X = df['clean_text']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf = True, max_df =10000) 
# just send in all your docs here 
X_train_trans=tfidf_vectorizer.fit_transform(X_train)
X_test_trans = tfidf_vectorizer.transform(X_test)

### Using Decision Tree

In [ ]:
forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(X_train_trans, y_train)


In [ ]:
y_pred_test_forest = forest.predict(X_test_trans)

In [ ]:
print(classification_report(y_test, y_pred_test_forest, target_names=lang_labels))

### TESTING

In [ ]:
test_df = pd.read_csv('test_set.csv')
test_df.head(1)

In [ ]:
sample_submision = pd.read_csv('sample_submission.csv')
sample_submision.head(1)

In [ ]:
#Chidex function
def coco(df):
    df['clean_text'] = df['text'].apply(remove_punctuation)
    
    # remove special characters, numbers and punctuations
    df['clean_text'] = df['clean_text'].str.replace("[^a-zA-Z#]", " ")
    
    return df['clean_text']

In [ ]:
call_test = coco(test_df)
call_test.head()

In [ ]:
X_ttest=tfidf_vectorizer.transform(call_test)

### Using Random Forest

In [ ]:
y_pred_call_test_for = forest.predict(X_ttest)

In [ ]:
y_pred_call_test_for

In [ ]:
y_pred_call_test_for = le.inverse_transform(y_pred_call_test_for)

In [ ]:
y_pred_call_test_for

### Creating Submission for Random Forest

In [ ]:
submissionfile2 = pd.DataFrame()

In [ ]:
submissionfile2['index']=test_df['index']

In [ ]:
submissionfile2.head()

In [ ]:
submissionfile2['lang_id'] = y_pred_call_test_for

In [ ]:
submissionfile2.head()

In [ ]:
submissionfile2.to_csv('deberesubmission_rf.csv',index= False)